In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score, KFold

In [15]:
filename = 'Data/Cleaned/post_feat_engineering.pkl'
with open(filename, mode='rb') as f:
    df = pickle.load(f)
df

,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_instate,med_income,percent_owned,percent_rented,med_value,med_hh_age,med_hh_residence,Proportion_citibike_rides,num_trees,med_rent,pred_year
GEOID,,,,,,,,,,,,,,,,,,,
36005000200,0.253498,-0.069683,-0.158731,0.045475,0.0,0.013086,0.054308,-0.001092,0.004986,0.039653,-0.097196,inf,0.011581,-0.026547,-0.016797,0.0,0.040706,0.000000,2020
36005000400,0.401586,0.111454,inf,-0.076004,0.0,-0.024307,-0.021735,0.022121,0.007430,0.016922,-0.039718,inf,-0.025644,-0.091655,-0.023268,0.0,0.099676,0.000000,2020
36005001600,0.206153,0.006892,0.000000,0.000000,0.0,0.016500,0.028395,-0.077970,-0.066064,0.009668,-0.185730,1.482936,0.004031,-0.021844,-0.050903,0.0,0.076535,0.000000,2020
36005001900,-0.231540,0.134766,0.000000,0.140154,0.0,-0.040426,0.062270,0.037248,0.195301,0.049870,-0.275184,0.090376,0.025174,-0.107038,-0.062557,0.0,0.000000,0.000000,2020
36005002000,0.266932,0.031160,inf,inf,0.0,0.007742,0.063012,-0.179134,-0.056130,-0.088320,-0.196227,inf,-0.004516,0.003948,-0.067870,0.0,0.000000,0.000000,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36085030301,-0.000015,-0.372741,inf,0.097769,0.0,-0.002331,0.085906,-0.037404,0.024439,0.132738,0.008724,0.001863,0.082799,0.000000,-0.041210,0.0,0.021161,0.016526,2026
36085030302,-0.058056,0.192520,0.000000,0.072049,0.0,-0.111086,0.045027,-0.021830,0.051280,0.090315,0.036488,-0.107441,0.113577,-0.024206,-0.048465,0.0,0.004528,0.022034,2026
36085031901,0.093828,-0.057249,0.000000,0.079193,0.0,0.034899,-0.012391,0.576102,0.118195,0.488474,0.071542,-0.022689,-0.014634,0.011404,-0.025731,0.0,0.054600,0.000000,2026


### Add Labels to Data

In [10]:
labels = pd.read_parquet('Data/Cleaned/labels.parquet')
labels

,GEOID,label,pred_year
0,36005001900,0,2020
7,36005002000,0,2020
13,36005002002,0,2020
14,36005003800,0,2020
23,36005004001,0,2020
...,...,...,...
19257,36081122704,0,2022
19260,36081142900,1,2022
19267,36085006700,0,2022
19273,36085008100,1,2022


In [23]:
df = df.merge(labels, on=['GEOID','pred_year'], how='left')
df

,GEOID,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_instate,...,percent_owned,percent_rented,med_value,med_hh_age,med_hh_residence,Proportion_citibike_rides,num_trees,med_rent,pred_year,label
0,36005000200,0.253498,-0.069683,-0.158731,0.045475,0.0,0.013086,0.054308,-0.001092,0.004986,...,-0.097196,inf,0.011581,-0.026547,-0.016797,0.0,0.040706,0.000000,2020,1.0
1,36005000400,0.401586,0.111454,inf,-0.076004,0.0,-0.024307,-0.021735,0.022121,0.007430,...,-0.039718,inf,-0.025644,-0.091655,-0.023268,0.0,0.099676,0.000000,2020,0.0
2,36005001600,0.206153,0.006892,0.000000,0.000000,0.0,0.016500,0.028395,-0.077970,-0.066064,...,-0.185730,1.482936,0.004031,-0.021844,-0.050903,0.0,0.076535,0.000000,2020,1.0
3,36005001900,-0.231540,0.134766,0.000000,0.140154,0.0,-0.040426,0.062270,0.037248,0.195301,...,-0.275184,0.090376,0.025174,-0.107038,-0.062557,0.0,0.000000,0.000000,2020,0.0
4,36005002000,0.266932,0.031160,inf,inf,0.0,0.007742,0.063012,-0.179134,-0.056130,...,-0.196227,inf,-0.004516,0.003948,-0.067870,0.0,0.000000,0.000000,2020,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6435,36085030301,-0.000015,-0.372741,inf,0.097769,0.0,-0.002331,0.085906,-0.037404,0.024439,...,0.008724,0.001863,0.082799,0.000000,-0.041210,0.0,0.021161,0.016526,2026,NaN
6436,36085030302,-0.058056,0.192520,0.000000,0.072049,0.0,-0.111086,0.045027,-0.021830,0.051280,...,0.036488,-0.107441,0.113577,-0.024206,-0.048465,0.0,0.004528,0.022034,2026,NaN
6437,36085031901,0.093828,-0.057249,0.000000,0.079193,0.0,0.034899,-0.012391,0.576102,0.118195,...,0.071542,-0.022689,-0.014634,0.011404,-0.025731,0.0,0.054600,0.000000,2026,NaN
6438,36085031902,-0.109198,-0.011178,inf,0.609157,0.0,0.016932,0.054347,-0.031721,0.007832,...,0.043006,-0.038730,0.090462,-0.022576,-0.033905,0.0,0.040412,0.000000,2026,NaN


### Split Data

In [24]:
# Isolate Pred Set
pred_set = df[df.pred_year=='2026']
train_set = df[df.pred_year!='2026']

In [26]:
# limit train set to vulnerable/gentrified tracts only TODO - also need to label pred set with vulnerable tracts
train_set = train_set[train_set.label!=0]
train_set

,GEOID,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_instate,...,percent_owned,percent_rented,med_value,med_hh_age,med_hh_residence,Proportion_citibike_rides,num_trees,med_rent,pred_year,label
0,36005000200,0.253498,-0.069683,-0.158731,0.045475,0.0,0.013086,0.054308,-0.001092,0.004986,...,-0.097196,inf,0.011581,-0.026547,-0.016797,0.0,0.040706,0.0,2020,1.0
2,36005001600,0.206153,0.006892,0.000000,0.000000,0.0,0.016500,0.028395,-0.077970,-0.066064,...,-0.185730,1.482936,0.004031,-0.021844,-0.050903,0.0,0.076535,0.0,2020,1.0
5,36005002300,-0.112276,-0.002417,1.220306,0.000000,0.0,-0.010294,-0.029477,0.338654,-0.024466,...,-0.339352,inf,0.008345,0.022090,-0.026120,0.0,0.021393,0.0,2020,1.0
7,36005002701,-0.017060,-0.210801,0.000000,0.000000,0.0,0.176933,0.120306,0.476796,-0.026033,...,0.000000,0.000000,0.014370,0.025328,-0.074691,0.0,0.202778,0.0,2020,1.0
8,36005002702,-0.060703,-0.007687,inf,0.000000,0.0,-0.005284,0.062451,inf,0.046983,...,-0.128667,inf,0.240898,0.050089,-0.034848,0.0,0.118925,0.0,2020,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,36085023100,0.018668,0.038200,-1.000000,-0.010049,0.0,-0.032483,0.097182,0.095700,-0.032220,...,-0.060171,inf,0.009994,-0.098034,-0.052272,0.0,0.072236,0.0,2022,1.0
4173,36085023900,0.035133,-0.142795,inf,-0.091138,0.0,0.139736,-0.034948,0.277655,-0.311116,...,-0.368635,inf,-0.082398,-0.044118,-0.047619,0.0,0.016913,0.0,2022,2.0
4175,36085024402,0.015707,inf,0.000000,-0.138261,0.0,-0.064973,0.058018,0.224444,0.031931,...,-0.059325,8.144025,0.068119,-0.023942,-0.035768,0.0,-0.008800,0.0,2022,2.0
4189,36085030301,-0.044640,0.006819,inf,0.312107,0.0,0.098457,0.033498,0.066675,0.010890,...,-0.153446,inf,0.006919,0.000000,-0.050000,0.0,0.047260,0.0,2022,2.0


In [3]:
# define feature set and split data

feat = ['percent_asian','percent_grad', 'percent_instate', 'percent_owned', 'percent_rented', 'med_value','med_hh_age',
       'med_hh_residence','Proportion_citibike_rides', 'num_trees', 'med_rent']

# split the data  ## TODO use cross validation
test = df_model.sample(300)
train = df_model[df_model.index.isin(test.index)==False]

y = df_model['gentrified']
y_train = train['gentrified']
y_test = test['gentrified']
X = df_model[feat]
X_train = train[feat]
X_test = test[feat]

X_train.shape

NameError: name 'df_model' is not defined

In [4]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_train_pred = logreg.predict(X_train)
y_test_pred = logreg.predict(X_test)

kf = KFold(n_splits=5, shuffle=True)
cv_scores = cross_val_score(logreg, X, y, cv=kf, scoring="accuracy")
print("CV Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))




# train_score = accuracy_score(y_train, y_train_pred)
# test_score =  accuracy_score(y_test, y_test_pred)
# print("Train Precision:", train_score)
# print("Test Precision:", test_score)


NameError: name 'X_train' is not defined

In [5]:
# Random Forest

forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

cv_scores = cross_val_score(forest, X, y, cv=kf,scoring="accuracy")
print("CV Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))


# train_score = accuracy_score(y_train, y_train_pred)
# test_score =  accuracy_score(y_test, y_test_pred)
# print("Train Precision:", train_score)
# print("Test Precision:", test_score)

NameError: name 'X_train' is not defined

In [6]:
# Create a confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test['gentrified'].unique())
disp.plot(cmap=plt.cm.Blues)

# Show the plot
plt.title("Confusion Matrix")
plt.show()

NameError: name 'y_test' is not defined